In [1]:
install.packages("glmnet")
install.packages("caret")
install.packages("dplyr")
install.packages("pROC")
install.packages("rsample")
install.packages("randomForest")


The downloaded binary packages are in
	/var/folders/91/4c3th19d31j6hvx23r6bb5b00000gn/T//RtmpqSvbFh/downloaded_packages

The downloaded binary packages are in
	/var/folders/91/4c3th19d31j6hvx23r6bb5b00000gn/T//RtmpqSvbFh/downloaded_packages

The downloaded binary packages are in
	/var/folders/91/4c3th19d31j6hvx23r6bb5b00000gn/T//RtmpqSvbFh/downloaded_packages

The downloaded binary packages are in
	/var/folders/91/4c3th19d31j6hvx23r6bb5b00000gn/T//RtmpqSvbFh/downloaded_packages

The downloaded binary packages are in
	/var/folders/91/4c3th19d31j6hvx23r6bb5b00000gn/T//RtmpqSvbFh/downloaded_packages

The downloaded binary packages are in
	/var/folders/91/4c3th19d31j6hvx23r6bb5b00000gn/T//RtmpqSvbFh/downloaded_packages


In [3]:
library(glmnet)
library(caret)
library(dplyr)
library(pROC)
library(rsample)
library(randomForest)

Loading required package: Matrix

Loaded glmnet 4.1-8

Loading required package: ggplot2

Loading required package: lattice


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following objects are masked from 'package:stats':

    cov, smooth, var


Warning message:
"package 'randomForest' was built under R version 4.3.3"
randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:dplyr':

    combine


The following object is masked from 'package:ggplot2':

    margin




# Import and format data

In [6]:
#read in the data
data <- read.csv("/Users/User/Documents/GitHub/DesktopDownthrottlingExperiment/post_experiment/Regression_2_(web_to_serviceable)/regression_tables/fieldwork_regression_table.csv")

#exclude cols and then scale 
exclude_columns <- c("identifier", "score_ellen", "score_aadya", "score_min", "serviceable_ellen", "serviceable_aadya", "serviceable_agg")

# Create a scaled version of the DataFrame excluding specified columns
scaled_data <- data %>%
  select(-one_of(exclude_columns)) %>%        # Exclude specified columns
  scale()                                      # Scale the remaining columns

# Convert the scaled data back to a DataFrame
scaled_data <- as.data.frame(scaled_data)

# Combine the scaled columns with the excluded columns from the original DataFrame
data <- cbind(data[exclude_columns], scaled_data)

#scale the score
data$scaled_score <- data$score_min/5
data$logistic_transformed_score <-  1 / (1 + exp(-data$scaled_score))
table(data$serviceable_agg)
trim_data <- data %>% select(-identifier,-score_min, - scaled_score,  -serviceable_agg, -score_ellen, -score_aadya, -serviceable_ellen, -serviceable_aadya)


 0  1 
59 31 

# Build a model

In [7]:
model <- lm(logistic_transformed_score ~ ., data = trim_data)
summary(model)


Call:
lm(formula = logistic_transformed_score ~ ., data = trim_data)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.065312 -0.012229 -0.000162  0.011500  0.057464 

Coefficients: (3 not defined because of singularities)
                                                     Estimate Std. Error
(Intercept)                                         6.240e-01  3.067e-03
IT01V_packetsRecieved_ellen                         1.633e-02  1.249e-01
IT01V_packetsRecieved_aadya                         8.661e-02  1.283e-01
IT01V_packetsLost_ellen                            -3.612e-03  4.690e-03
IT01V_packetsLost_aadya                             6.493e-03  5.253e-03
IT01V_frameWidth_ellen                              4.165e-02  7.176e-02
IT01V_frameWidth_aadya                              1.371e-02  1.462e-02
IT01V_frameHeight_ellen                                    NA         NA
IT01V_frameHeight_aadya                                    NA         NA
IT01V_totalFreezesDuration_ell

# Penalised regression

## Lasso/ Ridge

In [8]:
X <- model.matrix(logistic_transformed_score ~ ., data = trim_data)[,-1]
y <- trim_data$logistic_transformed_score

# Fit the lasso model (alpha = 1 for Lasso)
lasso_model <- glmnet(X, y, alpha = 1)

# Predict on training data
lasso_predictions <- predict(lasso_model, X)

# Calculate MSE for lasso model
mse_lasso <- mean((y - lasso_predictions)^2)

In [9]:
# Fit the ridge model (alpha = 0)
ridge_model <- glmnet(X, y, alpha = 0)

# Predict on training data
ridge_predictions <- predict(ridge_model, X)

# Calculate MSE for lasso model
mse_ridge <- mean((y - ridge_predictions)^2)

## AIC

In [10]:
fit_aic <- step(model)
summary(fit_aic)

aic_predictions <- predict(fit_aic, trim_data)
# par(bg = "white")
# options(repr.plot.width = 10, repr.plot.height = 6)
# plot(predictions, trim_data$logistic_transformed_score, 
#      main = "Prediction performance of AIC stepfit model", 
#      xlab = "Predicted values", ylab = "Recorded values")

# Calculate Mean Squared Error (MSE)
mse_aic <- mean((trim_data$logistic_transformed_score - aic_predictions)^2)

Start:  AIC=-609.11
logistic_transformed_score ~ IT01V_packetsRecieved_ellen + IT01V_packetsRecieved_aadya + 
    IT01V_packetsLost_ellen + IT01V_packetsLost_aadya + IT01V_frameWidth_ellen + 
    IT01V_frameWidth_aadya + IT01V_frameHeight_ellen + IT01V_frameHeight_aadya + 
    IT01V_totalFreezesDuration_ellen + IT01V_totalFreezesDuration_aadya + 
    IT01V_framesPerSecond_ellen + IT01V_framesPerSecond_aadya + 
    IT01V_bytesReceived_in_bits_s_ellen + IT01V_bytesReceived_in_bits_s_aadya + 
    IT01V_totalProcessingDelay_ellen + IT01V_totalProcessingDelay_aadya + 
    IT01V_jitter_ellen + IT01V_jitter_aadya + IT01V_jitterBufferDelay_emissions_ellen + 
    IT01V_jitterBufferDelay_emissions_aadya + IT01A_bytesReceived_in_bits_s_ellen + 
    IT01A_bytesReceived_in_bits_s_aadya + IT01A_jitterBufferDelay_emissions_ellen + 
    IT01A_jitterBufferDelay_emissions_aadya + OT01V_packetsSent_s_ellen + 
    OT01V_packetsSent_s_aadya + OT01V_bytesSent_in_bits_s_ellen + 
    OT01V_bytesSent_in_bits_s


Call:
lm(formula = logistic_transformed_score ~ IT01V_packetsLost_aadya + 
    IT01V_framesPerSecond_ellen + IT01V_framesPerSecond_aadya + 
    IT01V_totalProcessingDelay_ellen + IT01V_totalProcessingDelay_aadya + 
    IT01V_jitterBufferDelay_emissions_ellen + IT01A_jitterBufferDelay_emissions_ellen + 
    IT01A_jitterBufferDelay_emissions_aadya + OT01V_packetsSent_s_ellen + 
    OT01V_bytesSent_in_bits_s_ellen + OT01V_framesPerSecond_ellen + 
    OT01V_framesPerSecond_aadya + OT01V_totalPacketSendDelay_ellen + 
    RIV_roundTripTime_aadya + RIV_fractionLost_ellen + RIA_fractionLost_aadya + 
    RIA_roundTripTime_aadya + AP_totalPlayoutDelay_ellen, data = trim_data)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.055596 -0.012933  0.000302  0.014761  0.055469 

Coefficients:
                                         Estimate Std. Error t value Pr(>|t|)
(Intercept)                              0.623985   0.002613 238.829  < 2e-16
IT01V_packetsLost_aadya                

## BIC

In [11]:
fit_bic<- step(model, k = log(length(trim_data$logistic_transformed_score)))
#summary(fit_aic)

bic_predictions <- predict(fit_bic, trim_data)
# par(bg = "white")
# options(repr.plot.width = 10, repr.plot.height = 6)
# plot(predictions, trim_data$logistic_transformed_score, 
#      main = "Prediction performance of AIC stepfit model", 
#      xlab = "Predicted values", ylab = "Recorded values")

# Calculate Mean Squared Error (MSE)
mse_bic <- mean((trim_data$logistic_transformed_score - bic_predictions)^2)

Start:  AIC=-494.12
logistic_transformed_score ~ IT01V_packetsRecieved_ellen + IT01V_packetsRecieved_aadya + 
    IT01V_packetsLost_ellen + IT01V_packetsLost_aadya + IT01V_frameWidth_ellen + 
    IT01V_frameWidth_aadya + IT01V_frameHeight_ellen + IT01V_frameHeight_aadya + 
    IT01V_totalFreezesDuration_ellen + IT01V_totalFreezesDuration_aadya + 
    IT01V_framesPerSecond_ellen + IT01V_framesPerSecond_aadya + 
    IT01V_bytesReceived_in_bits_s_ellen + IT01V_bytesReceived_in_bits_s_aadya + 
    IT01V_totalProcessingDelay_ellen + IT01V_totalProcessingDelay_aadya + 
    IT01V_jitter_ellen + IT01V_jitter_aadya + IT01V_jitterBufferDelay_emissions_ellen + 
    IT01V_jitterBufferDelay_emissions_aadya + IT01A_bytesReceived_in_bits_s_ellen + 
    IT01A_bytesReceived_in_bits_s_aadya + IT01A_jitterBufferDelay_emissions_ellen + 
    IT01A_jitterBufferDelay_emissions_aadya + OT01V_packetsSent_s_ellen + 
    OT01V_packetsSent_s_aadya + OT01V_bytesSent_in_bits_s_ellen + 
    OT01V_bytesSent_in_bits_s

# Random forest

In [12]:
rf_model <- randomForest(logistic_transformed_score ~ ., data = trim_data)
# Make predictions
rf_predictions <- predict(rf_model, trim_data)
mse_rf <- mean((trim_data$logistic_transformed_score - rf_predictions)^2)

# Evaluate all the models

In [14]:
cat("MSE for AIC:", mse_aic, "\n")
cat("MSE for BIC:", mse_bic, "\n")
cat("MSE for lasso:", mse_lasso, "\n")
cat("MSE for ridge:", mse_ridge, "\n")

MSE for AIC: 0.0004846552 
MSE for BIC: 0.0005475032 
MSE for lasso: 0.0005733799 
MSE for ridge: 0.0009025711 
